# Gaussian Process Classifier

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid

from sklearn.datasets import make_moons, make_circles

from gp_utils import *

In [ ]:
class GPC:
    def __init__(self):
        """
        Initialize a Gaussian Process Classifier (GPC) object.
        The GPC is not fitted upon initialization.
        """
        self.theta = None
    
    def exp_kernel(self, X1, X2, theta):
        """
        Isotropic squared exponential kernel function.

        Args:
            X1: Array of m points (m x d).
            X2: Array of n points (n x d).
            theta: Kernel parameters (array-like).

        Returns:
            K: Gram matrix (m x n) computed using the squared exponential kernel.
        """

        sqdist = (
            np.sum(X1 ** 2, 1).reshape(-1, 1)
            + np.sum(X2 ** 2, 1)
            - 2 * np.dot(X1, X2.T)
        )
        return theta[1] ** 2 * np.exp(-0.5 / theta[0] ** 2 * sqdist)

    def K_(self, X, theta, diag_only=False, nu=1e-5):
        """
        Helper function to compute the covariance matrix K.

        Args:
            X: Data points (n x d).
            theta: Kernel parameters (array-like).
            diag_only: If True, compute only the diagonal elements of K.
            nu: Small constant to add to the diagonal of K for numerical stability.

        Returns:
            K: Covariance matrix (n x n) or its diagonal elements.
        """
        if diag_only:
            # Specific solution for isotropic
            # squared exponential kernel.
            return theta[1] ** 2 + nu
        else:
            return self.exp_kernel(X, X, theta) + nu * np.eye(X.shape[0])

    def W_(self, a):
        """
        Helper function to compute matrix W.

        Args:
            a: Logit values (n x 1).

        Returns:
            W: Diagonal matrix of weights (n x n).
        """
        r = sigmoid(a) * (1 - sigmoid(a))
        return np.diag(r.ravel())

    def posterior_mode(self, X, y, K_a, max_iter=10, tol=1e-9):
        """
        Computes the mode of posterior p(a|y).

        Args:
            X: Data points (n x d).
            y: Target values (n x 1).
            K_a: Covariance matrix K_a (n x n).
            max_iter: Maximum number of iterations for optimization.
            tol: Tolerance for convergence.

        Returns:
            a_h: Mode of the posterior p(a|y) (n x 1).
        """
        a_h = np.zeros_like(y)
        I = np.eye(X.shape[0])

        for i in range(max_iter):
            W = self.W_(a_h)
            Q_inv = np.linalg.pinv(I + W @ K_a)
            a_h_new = (K_a @ Q_inv).dot(y - sigmoid(a_h) + W.dot(a_h))
            a_h_diff = np.abs(a_h_new - a_h)
            a_h = a_h_new

            if not np.any(a_h_diff > tol):
                break

        return a_h

    def nll_fn(self, X, y):
        """
        Args:
            X: Data points (n x d).
            y: Target values (n x 1).

        Returns:
            nll: Negative log-likelihood function.
        """

        y = y.ravel()

        def nll(theta):
            K_a = self.K_(X, theta)
            K_a_inv = np.linalg.inv(K_a)

            # posterior mode depends on theta (via K)
            a_h = self.posterior_mode(X, y, K_a).ravel()
            W = self.W_(a_h)

            ll = (
                -0.5 * a_h.T.dot(K_a_inv).dot(a_h)
                - 0.5 * np.linalg.slogdet(K_a)[1]
                - 0.5 * np.linalg.slogdet(W + K_a_inv)[1]
                + y.dot(a_h)
                - np.sum(np.log(1.0 + np.exp(a_h)))
            )

            return -ll

        return nll
    
    def mean_var(self, X_test):
        """
        Computes the mean and variance of logits at points X_test
        given training data X, y, and kernel parameters theta.

        Args:
            X_test: Test data points (n_test x d).

        Returns:
            a_test_mu: Mean of logits (n_test x 1).
            a_test_var: Variance of logits (n_test x 1).
        """
        K_a = self.K_(self.X, self.theta)
        K_s = self.exp_kernel(self.X, X_test, self.theta)
        a_h = self.posterior_mode(self.X, self.y, K_a)

        W_inv = np.linalg.pinv(self.W_(a_h))
        R_inv = np.linalg.pinv(W_inv + K_a)

        a_test_mu = K_s.T.dot(self.y - sigmoid(a_h))
        # Compute variances only (= diagonal) instead of full covariance matrix
        a_test_var = self.K_(X_test, self.theta, diag_only=True) - np.sum((R_inv @ K_s) * K_s, axis=0).reshape(-1, 1)

        return a_test_mu, a_test_var

    def predict(self, X):
        """
        Computes the probability of points X being in class 1.
        GPC must be fitted before calling `predict`.
        """
        a_mu, a_var = self.mean_var(X)
        kappa = 1.0 / np.sqrt(1.0 + np.pi * a_var / 8)
        return sigmoid(kappa * a_mu)
    
    def predict_params(self, X_test):
        """
        Computes the mean and variance of logits at points X_test
        given training data X, y and kernel parameters theta.
        GPC must be fitted before calling `predict_params`.
        """
        
        K_a = self.K_(self.X, self.theta)
        K_s = self.exp_kernel(self.X, X_test, self.theta)
        a_h = self.posterior_mode(self.X, y, K_a)

        W_inv = np.linalg.inv(self.W_(a_h))
        R_inv = np.linalg.inv(W_inv + K_a)

        a_test_mu = K_s.T.dot(y - sigmoid(a_h))
        # Compute variances only (= diagonal) instead of full covariance matrix
        a_test_var = self.K_(X_test, self.theta, diag_only=True) - np.sum((R_inv @ K_s) * K_s, axis=0).reshape(-1, 1)

        return a_test_mu, a_test_var

    def fit(self, X, y):
        self.X = X
        self.y = y
        res = minimize(
            self.nll_fn(X, y),
            [1, 1],
            bounds=((1e-3, None), (1e-3, None)),
            method="L-BFGS-B",
        )
        self.theta = res.x
        
        print(f"Kernel parameters: {self.theta}, NLL = {res.fun:.3f}")

## Example A

In [ ]:
X, y = make_moons(200, noise=0.4, random_state=42)
y = y.reshape(-1, 1)

plot_data_2D(X, y)
plt.legend();

In [ ]:
gp = GPC()
gp.fit(X, y)

In [ ]:
grid_x, grid_y = np.mgrid[-4:4:200j, -4:4:200j]
grid = np.stack([grid_x, grid_y], axis=-1)

y_pred = gp.predict(grid.reshape(-1, 2)).reshape(*grid_x.shape)

In [ ]:
fig = plt.figure(figsize=(9, 7))
plot_pt_2D(grid_x, grid_y, y_pred)
plot_db_2D(grid_x, grid_y, y_pred, decision_boundary=0.5)
plot_data_2D(X, y)
plt.legend();

In [ ]:
a_test_var = gp.predict_params(grid.reshape(-1, 2))[1].reshape(*grid_x.shape)

plt.figure(figsize=(9, 7))
plt.contourf(grid_x, grid_y, a_test_var, alpha=0.3, cmap='viridis_r', levels=np.linspace(0, 15, 11))
plt.colorbar(label="Uncertainty")
plot_db_2D(grid_x, grid_y, y_pred, decision_boundary=0.5)
plot_data_2D(X, y)
plt.legend();

## Example B

In [ ]:
X, y = make_circles(n_samples=400, noise=0.25, random_state=42, factor=0.25)
y = y.reshape(-1, 1)

In [ ]:
plot_data_2D(X, y)

In [ ]:
gp = GPC()
gp.fit(X, y)

In [ ]:
grid_x, grid_y = np.mgrid[-1.5:2:200j, -1.5:2:200j]
grid = np.stack([grid_x, grid_y], axis=-1)


y_pred = gp.predict(grid.reshape(-1, 2)).reshape(*grid_x.shape)

In [ ]:
fig = plt.figure(figsize=(9, 7))
plot_pt_2D(grid_x, grid_y, y_pred)
plot_db_2D(grid_x, grid_y, y_pred, decision_boundary=0.5)
plot_data_2D(X, y)
plt.legend();

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
g = ax.plot_surface(grid_x, grid_y, y_pred, cmap="viridis")

ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$z$')
fig.colorbar(g, label="$p(y=1|x_1,x_2)$")
plt.show();

## Example C: Banknote authentication

In [ ]:
import pandas as pd

df = pd.read_csv("banknotes.csv", header=None)
df = df.sample(frac=1)

X = df[[0, 3]].to_numpy()
y = df[4].to_numpy().reshape(-1, 1)

In [ ]:
plot_data_2D(X, y)

In [ ]:
grid_x, grid_y = np.mgrid[-1.5:2:200j, -1.5:2:200j]
grid = np.stack([grid_x, grid_y], axis=-1)


y_pred = gp.predict(grid.reshape(-1, 2)).reshape(*grid_x.shape)

In [ ]:
gp = GPC()
gp.fit(X, y)

In [ ]:
grid_x, grid_y = np.mgrid[-1.5:2:200j, -1.5:2:200j]
grid = np.stack([grid_x, grid_y], axis=-1)


y_pred = gp.predict(grid.reshape(-1, 2)).reshape(*grid_x.shape)

In [ ]:
fig = plt.figure(figsize=(9, 7))
plot_pt_2D(grid_x, grid_y, y_pred)
plot_db_2D(grid_x, grid_y, y_pred, decision_boundary=0.5)
plot_data_2D(X, y)
plt.legend();

In [ ]:
a_test_var = gp.predict_params(grid.reshape(-1, 2))[1].reshape(*grid_x.shape)

plt.figure(figsize=(9, 7))
plt.contourf(grid_x, grid_y, a_test_var, alpha=0.3, cmap='viridis_r', levels=np.linspace(0, 15, 11))
plt.colorbar(label="Uncertainty")
plot_db_2D(grid_x, grid_y, y_pred, decision_boundary=0.5)
plot_data_2D(X, y)
plt.legend();
plt.savefig("bn_unc.pdf", bbox_inches='tight')